In [1]:
# Valimaki's research article:
# @inproceedings{valimaki1992real,
#   title={A real-time DSP implementation of a flute model},
#   author={Valimaki, V and Karjalainen, Matti and J{\'a}nosy, Zolt{\'a}n and Laine, Unto K},
#   booktitle={Acoustics, Speech, and Signal Processing, IEEE International Conference on},
#   volume={2},
#   pages={249--252},
#   year={1992},
#   organization={IEEE Computer Society}
# }

# Not Working for now 23.07.03

In [19]:
import numpy as np
import time
import soundfile as sf
from scipy.signal import lfilter

In [20]:
tube_length= 36
embochure = 20
sample_rate = 44100
nbr_de_sec = 0.5
inputGain = 0.372
total_samples = int(sample_rate*nbr_de_sec)

In [21]:
def noise(duration):
    y = (np.random.rand(duration,1)-0.5)*2
    return y

# def normalize(array):
#     normalized_array = (array - np.min(array)) / (np.max(array) - np.min(array))
#     normalized_array = (normalized_array-0.5) * 2 # -6 to 6
#     return normalized_array

def generate_envelope(attack_time, decay_time, sustain_level, release_time, sample_rate, total_samples):
    envelope = np.zeros(total_samples)

    attack_samples = int(attack_time * sample_rate)
    decay_samples = int(decay_time * sample_rate)
    release_samples = int(release_time * sample_rate)
    sustain_samples = total_samples - attack_samples - decay_samples - release_samples

    attack = np.linspace(0, 1, attack_samples)
    decay = np.linspace(1, sustain_level, decay_samples)
    sustain = np.full(sustain_samples, sustain_level)
    release = np.linspace(sustain_level, 0, release_samples)

    envelope[:attack_samples] = attack
    envelope[attack_samples:attack_samples+decay_samples] = decay
    envelope[attack_samples+decay_samples:attack_samples+decay_samples+sustain_samples] = sustain
    envelope[-release_samples:] = release

    return envelope

def dc_blocker(signal, cutoff_freq, fs):
    # Calculate filter coefficients　係数を計算
    t = 1.0 / (2.0 * np.pi * cutoff_freq)
    alpha = t / (t + (1.0 / fs))
    b = [1.0 - alpha, -1.0 + alpha]
    a = [1.0, -alpha]

    # Apply filter フィルターを適用
    filtered_signal = lfilter(b, a, signal)

    return filtered_signal



In [22]:
# Variables 変数

# Sample Rate サンプルレート
sample_rate = 44100

# Noise ノイズ
burst_noise = noise(total_samples)

# Noise Amplitude ノイズの振幅
noiseamp = 0.3

# Envelope エンベロープ
# Time in ms 時間（ミリ秒）
attack_time = 0.015
decay_time = 0.115
sustain_level = 0.0 # Staccato
release_time = 0.135

# Array for output 出力用配列
# Preallocate output array for efficiency 
# 効率のために出力配列を事前に割り当てる
output = np.empty(total_samples)  
delayTube1 = np.empty(total_samples)
delayTube2 = np.empty(total_samples)
delayEmb = np.empty(total_samples)
delayfb = np.empty(total_samples)
delayff = np.empty(total_samples)

# Feedback フィードバック
feedback1 = 0
feedback2 = 0

# Impulse Response Filter sample インパルス応答フィルターサンプル
delayfr = 0

# DC Blocker coefficient DCキラー係数
alpha = 0.5

# Measure Calculation Time 計算時間の計測
start = 0
end = 0


In [23]:
# Generate Envelope エンベロープの生成
envelope = generate_envelope(attack_time, decay_time, sustain_level, release_time, sample_rate, total_samples)

In [24]:
# Start calculation 計算開始

start = time.time()
for i in range(total_samples):
    instantSample = burst_noise[i]

    instantSample *= envelope[i]

    # + Feedback
    instantSample += feedback1

    # Input Gain
    instantSample *= inputGain

    # Delay-line Emb
    delayEmb[i] = instantSample
    if i > embochure:
        instantSample = delayEmb[i - embochure]

    # Nonlinearity
    instantSample = instantSample - (instantSample ** 3)

    # + Feedback
    instantSample += feedback2

    # Output1
    output1 = instantSample

    # Delay-line 1
    delayTube1[i] = instantSample
    if i > tube_length:
        instantSample = delayTube1[i - tube_length]

    # Fr
    instantSample_to_fr = instantSample
    instantSample_to_fr = instantSample * 0.11 + delayfr * (-0.89)
    delayfr = instantSample_to_fr
    output2 = instantSample_to_fr

    # Ff
    instantSample_to_ff = instantSample
    delayff[i] = instantSample_to_ff
    if i > 3:
        instantSample_to_ff = instantSample_to_ff * 0.25 + delayff[i-1] * 0.25 + delayff[i-2] * 0.25 + delayff[i-3] * 0.25
    instantSample = instantSample_to_ff

    # Delay-line 2
    delayTube2[i] = instantSample
    if i > tube_length:
        instantSample = delayTube2[i - tube_length]
    instantSample_start = instantSample
    
    # Fb
    instantSample_to_fb = instantSample
    delayfb[i] = instantSample_to_fb
    if i > 3:
        instantSample_to_fb = instantSample_to_fb * 0.25 + delayfb[i-1] * 0.25 + delayfb[i-2] * 0.25 + delayfb[i-3] * 0.25
    
    # feedback1
    feedback1 = instantSample_start * 0.1

    # feedback2
    feedback2 = instantSample_to_fb

    # Output
    output[i] = (output1 + output2) * 0.03162277660168379 # -30dB


# Apply DC Blocker DCキラーを適用
output = dc_blocker(output, 50, sample_rate)

# # Smooth the 100 first samples 最初の100サンプルを滑らかにする
# output[:100] = np.linspace(0, output[100], 100)

end = time.time()

À faire : supprimer la composante continue

In [25]:
# write output to file for listening
sf.write('output_Valimaki92.wav', output, sample_rate)

elapsed_time = end - start
print(elapsed_time)

0.1743619441986084
